```text
평가문항
    - 상세기준
1. 케글 데이터 분석 전과정이 성공적으로 진행되었는가?
    - 작성한 노트북을 케글에 제출했다.
2. 전처리, 학습과정 및 결과에 대한 설명이 시각화를 포함하여 체계적으로 진행되었는가?
    - 제출한 주피터노트북 파일이 케글 커널 환경에서도 에러 없이 동작하며, 전처리, 학습, 최적화 진행 과정이 체계적으로 기술되었다.
3. 회귀모델 예츠 정확도가 기준 이상 높게 나왔는가?
    - 다양한 피처 엔지니어링과 하이퍼 파라미터 튜닝 등의 최적화 기법을 통해 케클 리더보드의 Private score 기준 110000 이하의 점수를 보였다.
```

* 특이사항

로컬(MAC)에서 테스트르르 하다가 import error발생 (xgboost)
anaconda activation 중이었긴했지만 pip install --upgrade xgboost를 수행하는 바람에 제대로 된 셀 별 작업이 수행되지 않음

이에 노드(4-2)에서 작업한 코드를 하나의 셀에 작성하며 결과는 캡쳐본을 마크다운으로 명시


In [ ]:
# 시각화 그래프 준비
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 필요한 라이브러리 임포트 하기
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

# 내 입맛대로 데이터 준비하기
# 노드 예시
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'
# 로컬 예시
# data_dir = '../data/kaggles/kakr_housing'

train_data_path = join(data_dir, 'train.csv')
test_data_path = join(data_dir, 'test.csv') 

## train, test에 kaggle data load done
train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)

# 데이터 확인
train.head()

# datetime 컬럼의 값들을 6자리 정수로 변환하고 데이터 확인
train['date'] = train['date'].apply(lambda i: i[:6]).astype(int)
train.head()

# 학습데이터(train)에서 가격('price')를 빼기 위해 작업; 컬럼 삭제
# 가격은 변수 y에 저장함
y = train['price']
del train['price']

print(train.columns)

# 학습데이터(train)에서 id를 삭제하고 컬럼 확인
del train['id']

print(train.columns)

# test 데이터에서 데이터 전처리
## price 컬럼은 없으니 처리 생략하기
# [[YOUR CODE]]
# 데이터 살펴보기
test.head()
# 테스트 데이터의 'date' 컬럼 정수형으로 변경
test['date'] = test['date'].apply(lambda i: i[:6]).astype(int)
test.head()

# price 컬럼 작업 생략

# id 컬럼 삭제 및 나머지 컬럼 확인
del test['id']

print(test.columns)

# 타켓 데이터(y) 확인
y

# 데이터 분포도를 시각화하여 확인
"""
seaborn의 `kdeplot`을 활용해 `y`의 분포를 확인해주세요!
"""

#코드 작성
sns.kdeplot(y, shade=True)
plt.show()

# 좌편향 데이터를 로그 변환
y = np.log1p(y)
y

# 로그화된 데이터 분포를 시각화
sns.kdeplot(y)
plt.show()

# 전체 학습데이터 자료형 확인
train.info()

# 훈련 데이터셋과 검증 데이터셋으로 나눌 train_test_split() 호출
# RMSE 계산을 위한 모듈 호출
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# RMSE를 구하는 함수 선언
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))


# 네 가지 모델 가져오기
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# 모델 인스턴스 생성 및 결집
# random_state는 모델초기화나 데이터셋 구성에 사용되는 랜덤 시드값입니다. 
#random_state=None    # 이게 초기값입니다. 아무것도 지정하지 않고 None을 넘겨주면 모델 내부에서 임의로 선택합니다.  
random_state=2021        # 고정값을 2021로 변경해봄

gboost = GradientBoostingRegressor(random_state=random_state)
xgboost = XGBRegressor(random_state=random_state)
lightgbm = LGBMRegressor(random_state=random_state)
rdforest = RandomForestRegressor(random_state=random_state)

models = [gboost, xgboost, lightgbm, rdforest]

# 모델 이름 확인; __class__.__name__을 통해 가능
gboost.__class__.__name__

# 모델 별 학습 및 예측을 함수화
def get_scores(models, train, y):
    # 답안 작성
    df = {}

    for model in models:
        # 모델 이름 획득
        model_name = model.__class__.__name__

        # train, test 데이터셋 분리
        # random_state를 사용하여 고정하고 train과 test 셋의 비율은 8:2로 합니다.
        # [[YOUR CODE]]
        x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=random_state)

        # 모델 학습
        # [[YOUR CODE]]
        model.fit(x_train, y_train)
        
        # 예측
        # [[YOUR CODE]]
        y_pred = model.predict(x_test)

        # 예측 결과의 rmse값 저장
        df[model_name] = rmse(y_test, y_pred) # [[YOUR CODE]]
        
        # data frame에 저장
        score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)

    return df
# 모든 모델에서 score 획득해보기
get_scores(models, train, y)

# 다양한 하이퍼 파라미터 튜닝 테스트
## GridSearchCV 클래스 활용
from sklearn.model_selection import GridSearchCV

# xgboost 관련 하이퍼 파라미터 준비
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [1, 10],
}

# GridSearch할 함수 선언
def my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5):
    # GridSearchCV 모델로 초기화
    grid_model = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', \
                              cv=5, verbose=verbose, n_jobs=n_jobs)
    # 모델 fitting
    grid_model.fit(train, y)

    # 결과값 저장
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    # 데이터 프레임 생성
    results = pd.DataFrame(params)
    results['score'] = score
    
    # RMSLE 값 계산 후 정렬
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')

    return results

# GridSearch 성능확인
model = LGBMRegressor(random_state=random_state)
# n_jobs는 CPU 사용개수; 병렬처리;
results = my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5)



# sort_values()를 사용하여 RMSLE가 낮은 순서대로 정렬하기
# 위의 표를 `RMSLE`가 낮은 순서대로 정렬해주세요.
results.sort_values(by=['RMSLE'])

# 가장 좋은 조합인 max_depth=10, n_estimators=100 가지고 모델 학습 수행; 노드 기준
# 테스트는 8, 120으로 해봄
# 이후 예측값을 submission.csv 파일로 생성 후 kaggle에 제출
model = LGBMRegressor(max_depth=8, n_estimators=240, random_state=random_state)
model.fit(train, y)
prediction = model.predict(test)
prediction

## 로그변환값(RMSLE)을 원래 스케일로 되돌리기
prediction = np.expm1(prediction)
prediction

# sample_submission.csv 파일 가져오기
# 노드 예시 경로
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'
# data_dir = '../data/kaggles/kakr_housing'

submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission.head()

# sample dataframe의 price에 예측값 덮어씌우기
# kaggle 제출용 데이터 만들어짐
submission['price'] = prediction
submission.head()

# 위 데이터를 CSV 파일로 저장
# 모델 종류와 테스트일시를 파일에 명시
submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format(data_dir, 'lgbm', '20230712T0954')
submission.to_csv(submission_csv_path, index=False)
print(submission_csv_path)

![결과화면1](./images/1.png "결과화면1")</br>
![결과화면2](./images/2.png "결과화면2")</br>
![결과화면3](./images/3.png "결과화면3")</br>
![결과화면4](./images/4.png "결과화면4")</br>
</br>

csv는 다음과 같다.</br>
```text
    1. submission_lgbm_RMSLE_20230712.csv
        - random_state = 2021, max_depth = 10, n_estimaters = 100
    2. submission_lgbm_RMSLE_20230712T0741.csv
        - random_state = 2021, max_depth = 8, n_estimaters = 120
    3. submission_lgbm_RMSLE_20230712T0954.csv
        - random_state = 2021, max_depth = 8, n_estimaters = 240

```